In [258]:
import pandas as pd
import urllib.request
import os
from datetime import datetime,date
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import json
import yaml
from calendar import monthrange
import xarray as xr

#important
#before running this script, please git pull, then git checkout c3stest


def extract_dates_from_TSI():
    url='https://gerb.oma.be/tsi/C3S_RMIB_daily_TSI_composite_ICDR_v3_latest.txt'
    c=pd.read_csv(url,skiprows=128,delim_whitespace=True,header=None)
    return pd.Timestamp(str(c[3].iloc[0])),pd.Timestamp(str(c[3].iloc[-1]))
def extract_dates_icesheets():
    fname = '/Users/cxjo/Downloads/C3S_GrIS_RA_SEC_25km_vers4_2024-01-15.nc'
    nc = xr.open_dataset(fname)
    return pd.Timestamp(nc['time'].values[0]),pd.Timestamp(nc['time'].values[-1])
def extract_dates_massbalance():
    fname = '/Users/cxjo/Downloads/C3S_GMB_GRACE_vers4.nc'
    nc = xr.open_dataset(fname)
    return pd.Timestamp(nc['time'].values[0]),pd.Timestamp(nc['time'].values[-1])
def extract_dates_derived_glaciers(jfile):
    with open(jfile) as f:
        gen= yaml.safe_load(f)
    years = gen['labels']['hydrological_year'].keys()
    ymin=int(min(years))
    ymax=int(max(years))+1
    mmax=9
    dmax=30
    mmin=4
    dmin=1
    return pd.Timestamp(f'{ymin}-{mmin}-{dmin}'), pd.Timestamp(f'{ymax}-{mmax}-{dmax}')
def extract_dates_lake_levels():
    # need better way of estimating this
    fname = '/Users/cxjo/Downloads/C3S_LWL_N-AFRICA_VOLTA_altimetry_4.0_19921013_20221222_R20230126.nc'
    nc = xr.open_dataset(fname)
    return pd.Timestamp(nc['time'].values[0],unit='D'),pd.Timestamp(nc['time'].values[-1],unit='D')
def datemax2(row):
    row = row.dropna()
    ymax = int(max(row['year']))
    if 'month' in row.keys():
        mmax = int(max(row['month']))
    else:
        mmax=12
    xx,dmax=monthrange(ymax,mmax)
    datemax = pd.Timestamp(f'{ymax}-{mmax}-{dmax}')
    return datemax
def datemin2(row):
    row = row.dropna()
    ymin = int(min(row['year']))
    if 'month' in row.keys():
        mmin = int(min(row['month']))
    else:
        mmin=1
    xx,dmin=monthrange(ymin,mmin)
    datemin = pd.Timestamp(f'{ymin}-{mmin}-{dmin}')
    return datemin
def extract_dates_wv(jfilepath):
    f = open(jfilepath)
    # returns JSON object as 
    # a dictionary
    data = json.load(f)
    # print(data)
    df = pd.DataFrame(data)
    df['datemax'] = df.apply(datemax2,axis=1)
    df['datemin'] = df.apply(datemin2,axis=1)
    datemin = df['datemin'].min()
    datemax = df['datemax'].max()
    return datemin,datemax 
def check_time_agg(row):
    row = row.dropna()
    if 'time_aggregation' in row.keys():
        time_aggregation = row['time_aggregation'][0]
        # print('time_agg',time_aggregation)
        if time_aggregation in ['daily_average','daily_mean','day','day_average']: 
            time_agg = 'day'
            return time_agg
        elif time_aggregation == 'daily':
            time_agg='daily'
            return time_agg
        #note interim solution for 5-daily-composite...
        elif time_aggregation in [
            'monthly_average',
            '5_daily_composite',
            'monthly_mean',
            '27_days',
            'month',
            'monthly',
            'month_average',
            '10_day_average', # debatable..
            ]:
            time_agg = 'monthly'
            return time_agg
        else:
            # print(row)
            print('Could not determine time_agg')
            raise SystemExit
    elif 'period' in row.keys(): # applies to ice_sheets
        time_agg = 'period'
        return time_agg
    elif 'temporal_aggregation' in row.keys():
        time_aggregation = row['temporal_aggregation'][0]
        if time_aggregation in ['monthly','6-hourly']:
            time_agg='monthly'
        elif time_aggregation == 'daily':
            time_agg='daily'
        else:
            print('Error in temporal aggregation')
            raise SystemExit
        return time_agg
    else:
        if ('day' in row.keys()):
            time_agg='day'
            return time_agg
        elif  'nominal_day' in row.keys():
            time_agg='nominal_day'
            return time_agg
        else:
            time_agg = 'monthly'
            return time_agg
def compute_datemax(row):
 
    time_agg=check_time_agg(row) # check time aggregation of data in this row
    # print('time_agg',time_agg)
    if time_agg =='period':
        per_str=max(row['period'])
        ymax=int(per_str[5::])
        mmax=9
        dmax=30
    else: 
        ymax = int(max(row['year']))
        if 'month' in row.keys():
            mmax = int(max(row['month']))
        else:
            mmax=12
        xx,ndays = monthrange(ymax,mmax)
        if time_agg in ['day','nominal_day']:
            # print(row[time_agg])
            dmax = int(max(row[time_agg])) 
        elif time_agg in ['daily']:
            dmax = int(max(row['day'])) 
        else:
            dmax = ndays # last day of month
        if dmax >ndays:
            print('Beware error in allowed dates...')
            # print(row)
            dmax=ndays

    datemax = pd.Timestamp(f'{ymax}-{mmax}-{dmax}')
    return datemax
def compute_datemin(row):
    time_agg=check_time_agg(row) # check time aggregation of data in this row
    if time_agg =='period':
        per_str=max(row['period'])
        ymin=int(per_str[0:4])
        mmin=10
        dmin=1
    else:
        ymin = int(min(row['year'])) 
        if 'month' in row.keys():
            mmin = int(min(row['month']))
        else:
            mmin=1
        if time_agg in ['day','nominal_day']:
            dmin = int(min(row[time_agg])) 
        elif time_agg in ['daily']:
            dmin = int(min(row['day'])) 
        else:
            dmin = 1 # first day of month
        xx,ndays = monthrange(ymin,mmin)     
        if dmin>ndays:
            print('Beware error in allowed dates...')
            # print(row)
            dmin=1
    datemin = pd.Timestamp(f'{ymin}-{mmin}-{dmin}')
    return datemin
def calc_dateminmax_from_cds_form(jfilepath,ecv):
    # Opening JSON file
    f = open(jfilepath)
    # returns JSON object as 
    # a dictionary
    data = json.load(f)
    # print(data)
    df = pd.DataFrame(data)
    # display(df)
    # print(df.keys())
    # print(len(df))
    
    # find records where dates cannot be defined
    if 'sensor_and_algorithm' in df.keys():        
        lst_erase=[]
        for i in range(len(df)):
            if (df['sensor_and_algorithm'][i][0]=='merged_obs4mips'): lst_erase.append(i)
        # now .drop these problematic rows
        for i in lst_erase:
            df=df.drop(lst_erase)
    if ecv == 'Earth Radiation Budget':        
        lst_erase=[]
        for i in range(len(df)):
            if (df['variable'][i][0]=='total_solar_irradiance'): lst_erase.append(i) # this info is read from the dataset itself
        # now .drop these problematic rows
        for i in lst_erase:
            df=df.drop(lst_erase)
    # for i in range(len(df)):
    #     print(df.loc[i])
    #     if ('year' not in df[i]): lst_erase.append(i)

    df['datemax'] = df.apply(compute_datemax,axis=1)
    df['datemin'] = df.apply(compute_datemin,axis=1)
    datemin = df['datemin'].min()
    datemax = df['datemax'].max()
    return datemin,datemax

with open('config.yml') as f:
    conf= yaml.safe_load(f)

cds_form_dir=conf['cds_form_dir']


datesbeg = {}
datesend = {}

for ecv in conf['ECV']:
    print(ecv)
    entries = conf['ECV'][ecv]['entry']
    print(entries)
    datemin_list = []
    datemax_list = []
    if ecv in ['Earth Radiation Budget']: 
        datemin,datemax = extract_dates_from_TSI()
        datemin_list.append(datemin)
        datemax_list.append(datemax)
    for entry in entries:
        jfilepath=f'{cds_form_dir}{entry}/constraints.json'
        print(jfilepath)
        if jfilepath == '/Users/cxjo/Documents/cds-forms-c3s/satellite-ice-sheet-elevation-change/constraints.json':
            datemin,datemax = extract_dates_icesheets()
            datemin_list.append(datemin)
            datemax_list.append(datemax)
        elif jfilepath == '/Users/cxjo/Documents/cds-forms-c3s/satellite-ice-sheet-mass-balance/constraints.json':
            datemin,datemax = extract_dates_massbalance()
            datemin_list.append(datemin)
            datemax_list.append(datemax)
        elif jfilepath == '/Users/cxjo/Documents/cds-forms-c3s/derived-gridded-glacier-mass-change//constraints.json':
            jfile = f'{cds_form_dir}{entry}/generate.yaml'
            datemin,datemax = extract_dates_derived_glaciers(jfile)
            datemin_list.append(datemin)
            datemax_list.append(datemax)
        elif jfilepath == '/Users/cxjo/Documents/cds-forms-c3s/insitu-glaciers-extent/constraints.json':    
            datemin= pd.Timestamp('1990-01-01') # http://www.glims.org/rgi_user_guide/06_dataset_summary.html
            datemax= pd.Timestamp('2010-12-31')
            datemin_list.append(datemin)
            datemax_list.append(datemax)
        elif jfilepath =='/Users/cxjo/Documents/cds-forms-c3s/satellite-total-column-water-vapour-ocean/constraints.json':
            # temporal aggregation is messed up. does not have the same meaning as other datasets
            # monthly should be yearly
            # 6-hourly should be monthly
            # need to write a special function that accounts for this
            datemin,datemax = extract_dates_wv(jfilepath)
            datemin_list.append(datemin)
            datemax_list.append(datemax)
        elif jfilepath == '/Users/cxjo/Documents/cds-forms-c3s/satellite-lake-water-level/constraints.json':
            datemin,datemax = extract_dates_lake_levels()
            datemin_list.append(datemin)
            datemax_list.append(datemax)
        else:
            datemin,datemax = calc_dateminmax_from_cds_form(jfilepath,ecv)
            # print(ecv,datemin_list,datemax_list)
            datemin_list.append(datemin)
            datemax_list.append(datemax)
            # print(ecv,datemin_list,datemax_list)

    # now get the max and min per ECV, accounting for all products
    datemin_list = np.array(datemin_list)
    datemax_list = np.array(datemax_list)
    datesbeg[ecv] = np.min(datemin_list)
    datesend[ecv] = np.max(datemax_list)


Aerosols
['satellite-aerosol-properties']
/Users/cxjo/Documents/cds-forms-c3s/satellite-aerosol-properties/constraints.json
Beware error in allowed dates...
Beware error in allowed dates...
Albedo
['satellite-albedo']
/Users/cxjo/Documents/cds-forms-c3s/satellite-albedo/constraints.json
Greenhouse Gases
['satellite-carbon-dioxide', 'satellite-methane']
/Users/cxjo/Documents/cds-forms-c3s/satellite-carbon-dioxide/constraints.json
/Users/cxjo/Documents/cds-forms-c3s/satellite-methane/constraints.json
Clouds
['satellite-cloud-properties']
/Users/cxjo/Documents/cds-forms-c3s/satellite-cloud-properties/constraints.json
Earth Radiation Budget
['satellite-earth-radiation-budget']
/Users/cxjo/Documents/cds-forms-c3s/satellite-earth-radiation-budget/constraints.json
Fire
['satellite-fire-burned-area', 'satellite-fire-radiative-power']
/Users/cxjo/Documents/cds-forms-c3s/satellite-fire-burned-area/constraints.json
/Users/cxjo/Documents/cds-forms-c3s/satellite-fire-radiative-power/constraints.jso

AttributeError: 'Timestamp' object has no attribute 'astype'

In [257]:

ecv_pd = pd.DataFrame([datesbeg,datesend],index=['DateBeg','DateEnd']).T
ecv_pd['DateBeg'] = ecv_pd['DateBeg'].astype('datetime64[D]')
print(ecv_pd)

                                               DateBeg  \
Aerosols                 1995-06-01 00:00:00.000000000   
Albedo                   1981-09-20 00:00:00.000000000   
Greenhouse Gases         2002-10-01 00:00:00.000000000   
Clouds                   1982-01-01 00:00:00.000000000   
Earth Radiation Budget   1979-01-01 00:00:00.000000000   
Fire                     2001-01-01 00:00:00.000000000   
Ice Sheets               1975-04-01 00:00:00.000000000   
Glaciers                 1990-01-01 00:00:00.000000000   
Upper-air Water Vapour   1988-01-31 00:00:00.000000000   
LAI                      1981-09-20 00:00:00.000000000   
FAPAR                    1981-09-20 00:00:00.000000000   
Lakes                    1992-10-13 13:52:00.002174464   
Land Cover               1992-01-01 00:00:00.000000000   
Ocean Colour             1997-09-04 00:00:00.000000000   
Ozone                    1970-04-01 00:00:00.000000000   
Precipitation            1979-01-01 00:00:00.000000000   
Sea Ice       

datemin: 1992-01-01 00:00:00 datemax: 2023-07-01 00:00:00
